## **Import necessary libraries**

In [1]:
from openpyxl import Workbook
from evaluate import load

import os
import csv
import lmstudio as lms

/home/syahputra/miniconda3/envs/lmstudio/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## **Define paths to dataset and output**

In [2]:
image_folder = "dataset/images"
gt_folder = "dataset/ground-truth"
output_excel = "result/prediction.xlsx"

## **Model initialization**

In [3]:
model = lms.llm("google/gemma-3-4b")
# model = lms.llm("google/gemma-3-12b:gemma-3-12b-it")

## **Excel file setup**

In [4]:
wb = Workbook()
ws = wb.active
ws.title = "plate number predictions"
ws.append(["image", "ground_truth", "prediction", "CER_score"])

## **Define CER score calculation function**

In [5]:
def calculate_cer(predictions, ground_truths):
    cer_metric = load("cer")
    result = cer_metric.compute(predictions=predictions, references=ground_truths)
    return result

## **List all image files**

In [6]:
ekstensi_gambar = [".jpg", ".jpeg", ".png"]
files = os.listdir(image_folder)
gambar_files = [f for f in files if os.path.splitext(f)[1].lower() in ekstensi_gambar]
gambar_files.sort()

print(f"Images found: {len(gambar_files)}")

Images found: 20


## **Main prediction loop**

In [7]:
results = []

for gambar in gambar_files:
    nama = os.path.splitext(gambar)[0]
    image_path = os.path.join(image_folder, gambar)
    txt_path = os.path.join(gt_folder, f"{nama}.txt")

    if not os.path.exists(txt_path):
        print(f"Cannot find ground truth for {gambar}, skipping.")
        continue

    with open(txt_path, "r") as f:
        ground_truth = f.read().strip()

    image_handle = lms.prepare_image(image_path)

    chat = lms.Chat()
    chat.add_user_message(
        "What is the license plate number shown in this image? Respond only with the license plate characters, without any spaces, or punctuation. Do not include the expiration date.",
        images=[image_handle]
    )

    prediction_result = model.respond(chat)
    prediction = prediction_result.content.strip()
    cer_score = calculate_cer([prediction], [ground_truth])

    ws.append([gambar, ground_truth, prediction, round(cer_score, 3)])
    results.append([gambar, ground_truth, prediction, round(cer_score, 3)])
    print(f"{gambar} → GT: {ground_truth}, Pred: {prediction}, CER: {cer_score:.3f}")
    print("-" * 80)

sample1.jpg → GT: N1365AA0, Pred: N1365AA0, CER: 0.000
--------------------------------------------------------------------------------
sample10.jpg → GT: L7540GP, Pred: L7540GP, CER: 0.000
--------------------------------------------------------------------------------
sample11.jpg → GT: N1112BU, Pred: 1112BU, CER: 0.143
--------------------------------------------------------------------------------
sample12.jpg → GT: L1707WE, Pred: 1707WE, CER: 0.143
--------------------------------------------------------------------------------
sample13.jpg → GT: L1792AAQ, Pred: L1792AAQ, CER: 0.000
--------------------------------------------------------------------------------
sample14.jpg → GT: L1952VG, Pred: 1952VG, CER: 0.143
--------------------------------------------------------------------------------
sample15.jpg → GT: L1213HX, Pred: 1213HX, CER: 0.143
--------------------------------------------------------------------------------
sample16.jpg → GT: L1983ZP, Pred: 1983ZP, CER: 0.143
---

## **Save excel file**

In [8]:
wb.save(output_excel)
print(f"Results saved to: {output_excel}")

Results saved to: result/prediction.xlsx


In [9]:
output_csv = "result/prediction.csv"
os.makedirs(os.path.dirname(output_csv), exist_ok=True)

with open(output_csv, mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["image", "ground_truth", "prediction", "CER_score"])
    writer.writerows(results)

print(f"Results saved to: {output_csv}")


Results saved to: result/prediction.csv
